In [1]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Input,Activation,Flatten
from tensorflow.keras.regularizers import l1,l2
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np

filters_per_conv_layer = [16,16]
neurons_per_dense_layer = [32,64]
batch_size=250

x = x_in = Input((7,7,1))

for i,f in enumerate(filters_per_conv_layer):
    print( ('Adding convolutional block {} with N={} filters').format(i,f) )
    x = Conv2D(int(f), kernel_size=(2,2), strides=(1,1), kernel_initializer='lecun_uniform',padding="same", kernel_regularizer=l1(0.0001), use_bias=False,
               name='conv_{}'.format(i))(x) 
    x = BatchNormalization(name='bn_conv_{}'.format(i))(x)
    x = Activation('relu',name='conv_act_%i'%i)(x)


x = MaxPooling2D(pool_size = (2,2))(x)

x = Flatten()(x)

for i,n in enumerate(neurons_per_dense_layer):
    print( ('Adding dense block {} with N={} neurons').format(i,n) )
    x = Dense(n,kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001),name='dense_%i'%i, use_bias=False)(x)
    x = BatchNormalization(name='bn_dense_{}'.format(i))(x)
    x = Activation('relu',name='dense_act_%i'%i)(x)
x = Dense(2,name='output_dense')(x)
x_out = Activation('softmax',name='output_softmax')(x)

model = Model(inputs=[x_in], outputs=[x_out], name='keras_baseline')
model.compile(loss='categorical_crossentropy',
                      optimizer=tf.keras.optimizers.Adam(learning_rate=1.578e-5),
                      metrics=['accuracy'],)


2023-02-26 22:11:20.708154: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-26 22:11:20.708172: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Adding convolutional block 0 with N=16 filters
Adding convolutional block 1 with N=16 filters
Adding dense block 0 with N=32 neurons
Adding dense block 1 with N=64 neurons


2023-02-26 22:11:21.815101: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-26 22:11:21.815118: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-26 22:11:21.815127: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ej-ubt): /proc/driver/nvidia/version does not exist
2023-02-26 22:11:21.815245: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model.summary()

Model: "keras_baseline"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 7, 7, 1)]         0         
                                                                 
 conv_0 (Conv2D)             (None, 7, 7, 16)          64        
                                                                 
 bn_conv_0 (BatchNormalizati  (None, 7, 7, 16)         64        
 on)                                                             
                                                                 
 conv_act_0 (Activation)     (None, 7, 7, 16)          0         
                                                                 
 conv_1 (Conv2D)             (None, 7, 7, 16)          1024      
                                                                 
 bn_conv_1 (BatchNormalizati  (None, 7, 7, 16)         64        
 on)                                                

## generate CNN model io stream, non-quantized

In [3]:
from tensorflow_model_optimization.sparsity.keras import strip_pruning
model  = strip_pruning(model)

In [4]:
import hls4ml
import plotting
import os
os.environ['PATH'] = '/opt/Xilinx/Vivado/2019.2/bin:' + os.environ['PATH']

hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')

# Set the precision and reuse factor for the full model
hls_config['Model']['Precision'] = 'ap_fixed<16,6>'
hls_config['Model']['ReuseFactor'] = 1

# Create an entry for each layer, here you can for instance change the strategy for a layer to 'resource' 
# or increase the reuse factor individually for large layers.
# In this case, we designed the model to be small enough for a fully parallel implementation 
# so we use the latency strategy and reuse factor of 1 for all layers.
for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Strategy'] = 'Resource'
    hls_config['LayerName'][Layer]['ReuseFactor'] = 2
#If you want best numerical performance for high-accuray models, while the default latency strategy is faster but numerically more unstable
hls_config['LayerName']['output_softmax']['Strategy'] = 'Stable'

cfg = hls4ml.converters.create_config(backend='Vivado')
cfg['IOType']     = 'io_parallel'
cfg['HLSConfig']  = hls_config
cfg['KerasModel'] = model
cfg['OutputDir']  = 'cnn_parallel_resource_7x7'
cfg['Part'] = 'xcvu9p-flga2104-2L-e'
  
hls_model = hls4ml.converters.keras_to_hls(cfg)
hls_model.compile()

Interpreting Model
Topology:
Layer name: input_1, layer type: Input
Layer name: conv_0, layer type: Conv2D
  -> Activation (linear), layer name: conv_0
Layer name: bn_conv_0, layer type: BatchNormalization
Layer name: conv_act_0, layer type: Activation
Layer name: conv_1, layer type: Conv2D
  -> Activation (linear), layer name: conv_1
Layer name: bn_conv_1, layer type: BatchNormalization
Layer name: conv_act_1, layer type: Activation
Layer name: max_pooling2d, layer type: MaxPooling2D
Layer name: dense_0, layer type: Dense
  -> Activation (linear), layer name: dense_0
Layer name: bn_dense_0, layer type: BatchNormalization
Layer name: dense_act_0, layer type: Activation
Layer name: dense_1, layer type: Dense
  -> Activation (linear), layer name: dense_1
Layer name: bn_dense_1, layer type: BatchNormalization
Layer name: dense_act_1, layer type: Activation
Layer name: output_dense, layer type: Dense
  -> Activation (linear), layer name: output_dense
Layer name: output_softmax, layer type:

/home/ej/workspace/hls4ml/hls4ml/hls4ml/converters/__init__.py:15: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


Done


In [5]:
hls_model.build(csim=False, synth=True, vsynth=True)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2019.2/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'ej' on host 'ej-ubt' (Linux_x86_64 version 5.4.0-139-generic) on Sun Feb 26 22:11:28 PST 2023
INFO: [HLS 200-10] On os Ubuntu 18.04.6 LTS
INFO: [HLS 200-10] In directory '/home/ej/workspace/gluon_tagger_algo/cnn_parallel_resource_7x7'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Creating and opening project '/home/ej/workspace/gluon_tagger_algo/cnn_parallel_resource_7x7/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject.cpp' to the project
INFO: [HLS 200-10] Adding test bench file 'myproject_test.cpp' to the projec

INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_mult>' into 'nnet::conv_2d_cl<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' (firmware/nnet_utils/nnet_conv2d_resource.h:97->firmware/nnet_utils/nnet_conv2d.h:73) automatically.
INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config6_mult>' into 'nnet::conv_2d_cl<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config6>' (firmware/nnet_utils/nnet_conv2d_resource.h:97->firmware/nnet_utils/nnet_conv2d.h:73) automatically.
INFO: [XFORM 203-602] Inlining function 'nnet::max<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 4>' into 'nnet::pool_op<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 4, (nnet::Pool_Op)0>' (firmware/nnet_utils/

INFO: [HLS 200-489] Unrolling loop 'Result' (firmware/nnet_utils/nnet_dense_resource.h:98) in function 'nnet::dense_resource<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config16>' completely with a factor of 64.
INFO: [HLS 200-489] Unrolling loop 'Loop-1' (firmware/nnet_utils/nnet_activation.h:71) in function 'nnet::relu<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, relu_config15>' completely with a factor of 32.
INFO: [HLS 200-489] Unrolling loop 'InitAccum' (firmware/nnet_utils/nnet_dense_resource.h:58) in function 'nnet::dense_resource<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config12>' completely with a factor of 32.
INFO: [HLS 200-489] Unrolling loop 'MultLoop' (firmware/nnet_utils/nnet_dense_resource.h:73) in function 'nnet::dense_resource<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mod

INFO: [XFORM 203-101] Partitioning array 'data_buf.i'  in dimension 2 completely.
INFO: [XFORM 203-101] Partitioning array 'acc.V' (firmware/nnet_utils/nnet_conv2d_resource.h:32) in dimension 2 completely.
INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_mult>' into 'nnet::conv_2d_cl<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' (firmware/nnet_utils/nnet_conv2d_resource.h:97->firmware/nnet_utils/nnet_conv2d.h:73) automatically.
INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config6_mult>' into 'nnet::conv_2d_cl<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config6>' (firmware/nnet_utils/nnet_conv2d_resource.h:97->firmware/nnet_utils/nnet_conv2d.h:73) automatically.
INFO: [X

INFO: [XFORM 203-11] Balancing expressions in function 'nnet::conv_2d_cl<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config6>' (firmware/nnet_utils/nnet_conv2d_resource.h:36:13)...512 expression(s) balanced.
INFO: [HLS 200-111] Finished Pre-synthesis Time (s): cpu = 00:15:24 ; elapsed = 00:15:30 . Memory (MB): peak = 3709.559 ; gain = 3259.066 ; free physical = 23575 ; free virtual = 34625

the outer loop is not a perfect loop.

the outer loop is not a perfect loop.
INFO: [XFORM 203-531] Rewinding loop 'ReuseLoop' (firmware/nnet_utils/nnet_dense_resource.h:64) in function 'dense_resource<ap_fixed,ap_fixed<16,6,5,3,0>,config20>'.
INFO: [XFORM 203-531] Rewinding loop 'ReuseLoop' (firmware/nnet_utils/nnet_dense_resource.h:64) in function 'dense_resource<ap_fixed,ap_fixed<16,6,5,3,0>,config16>'.
INFO: [XFORM 203-531] Rewinding loop 'ReuseLoop' (firmware/nnet_utils/nnet_dense_resource.h:64) in function 'dense_resource<ap_fixed,ap_fixed<16,

...

INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_60_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_61_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_62_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_63_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_64_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_65_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_66_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_67_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_68_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_69_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285]

INFO: [RTMG 210-285] Implementing FIFO 'layer12_out_0_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer12_out_1_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer12_out_2_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer12_out_3_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer12_out_4_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer12_out_5_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer12_out_6_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer12_out_7_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer12_out_8_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer12_out_9_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implement

INFO: [RTMG 210-285] Implementing FIFO 'layer16_out_20_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer16_out_21_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer16_out_22_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer16_out_23_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer16_out_24_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer16_out_25_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer16_out_26_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer16_out_27_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer16_out_28_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer16_out_29_V_U(fifo_w16_d2_A)' using Shift Registers.
INFO: [RTMG 210-285]

INFO: [RTMG 210-285] Implementing FIFO 'layer19_out_40_V_U(fifo_w15_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer19_out_41_V_U(fifo_w15_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer19_out_42_V_U(fifo_w15_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer19_out_43_V_U(fifo_w15_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer19_out_44_V_U(fifo_w15_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer19_out_45_V_U(fifo_w15_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer19_out_46_V_U(fifo_w15_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer19_out_47_V_U(fifo_w15_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer19_out_48_V_U(fifo_w15_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer19_out_49_V_U(fifo_w15_d2_A)' using Shift Registers.
INFO: [RTMG 210-285]

{'CSynthesisReport': {'EstimatedClockPeriod': '4.374',
  'BestLatency': '855',
  'WorstLatency': '858',
  'IntervalMin': '442',
  'IntervalMax': '442',
  'BRAM_18K': '1738',
  'DSP48E': '3938',
  'FF': '130748',
  'LUT': '907690',
  'URAM': '0',
  'AvailableBRAM_18K': '4320',
  'AvailableDSP48E': '6840',
  'AvailableFF': '2364480',
  'AvailableLUT': '1182240',
  'AvailableURAM': '960'}}

In [6]:
hls4ml.report.read_vivado_report('cnn_parallel_resource_7x7')

Found 1 solution(s) in cnn_parallel_resource_7x7/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject'
* Date:           Sun Feb 26 22:57:16 2023

* Version:        2019.2 (Build 2704478 on Wed Nov 06 22:10:23 MST 2019)
* Project:        myproject_prj
* Solution:       solution1
* Product family: virtexuplus
* Target device:  xcvu9p-flga2104-2L-e


== Performance Estimates
+ Timing: 
    * Summary: 
    +--------+---------+----------+------------+
    |  Clock |  Target | Estimated| Uncertainty|
    +--------+---------+----------+------------+
    |ap_clk  | 5.00 ns | 4.374 ns |   0.62 ns  |
    +--------+---------+----------+------------+

+ Latency: 
    * Summary: 
    +---------+---------+----------+----------+-----+-----+----------+
    |  Latency (cycles) |  Latency (absolute) |  Interval | Pipeline |
    |   min   |   max   |    min   |    max   | min | max |   Type   |
    +---------+---------+---

In [7]:
* Summary: parallel_7x7
    +---------+---------+----------+----------+-----+-----+----------+
    |  Latency (cycles) |  Latency (absolute) |  Interval | Pipeline |
    |   min   |   max   |    min   |    max   | min | max |   Type   |
    +---------+---------+----------+----------+-----+-----+----------+
    |      855|      858| 4.275 us | 4.290 us |  442|  442| dataflow |
    +---------+---------+----------+----------+-----+-----+----------+
    
* Summary: stream_7x7
    +---------+---------+----------+----------+-----+-----+----------+
    |  Latency (cycles) |  Latency (absolute) |  Interval | Pipeline |
    |   min   |   max   |    min   |    max   | min | max |   Type   |
    +---------+---------+----------+----------+-----+-----+----------+
    |      395|      396| 1.975 us | 1.980 us |  129|  385| dataflow |
    +---------+---------+----------+----------+-----+-----+----------+
    
* Summary: parallel_9x9
    +---------+---------+----------+----------+-----+-----+----------+
    |  Latency (cycles) |  Latency (absolute) |  Interval | Pipeline |
    |   min   |   max   |    min   |    max   | min | max |   Type   |
    +---------+---------+----------+----------+-----+-----+----------+
    |     1822|     1825| 9.110 us | 9.125 us |  892|  892| dataflow |
    +---------+---------+----------+----------+-----+-----+----------+
    
* Summary: stream_9x9
    +---------+---------+----------+----------+-----+-----+----------+
    |  Latency (cycles) |  Latency (absolute) |  Interval | Pipeline |
    |   min   |   max   |    min   |    max   | min | max |   Type   |
    +---------+---------+----------+----------+-----+-----+----------+
    |      633|      634| 3.165 us | 3.170 us |  201|  601| dataflow |
    +---------+---------+----------+----------+-----+-----+----------+


SyntaxError: invalid syntax (3037950879.py, line 1)